In [1]:
import becquerel as bq
from becquerel import Spectrum
from becquerel.tools import Isotope, IsotopeQuantity, NeutronIrradiation
import datetime
import numpy as np
import scipy.integrate as integrate
import math as m
import importlib
import pandas as pd
import sys
import os
import matplotlib.pyplot as plt
sys.path.insert(0,r"C:\Users\Daniel\Desktop\Radwatch\radwatch-analysis")

In [2]:
import PF
import naa_isotope_analyzer as naa
import analysis_methods as am
import ROI
importlib.reload(PF)
importlib.reload(naa)
importlib.reload(am)
importlib.reload(ROI)

<module 'ROI' from 'C:\\Users\\Daniel\\Desktop\\Radwatch\\radwatch-analysis\\ROI.py'>

In [3]:
def get_isotopes(energies):
    hl_cut = 3600 #in becq.fetch decay radiation
    br_cut =4.0
    df = naa.naa_isotope_analyzer(energies.tolist(),hl_cut,br_cut)
    
    return df

In [4]:
#use the ecal and bgspec from google drive
#implement choosing between relevant ecal and bg for diff sample groups
e_cal = np.loadtxt(r"C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\11-19-2019-Calibration.Ent")
bgspec =  Spectrum.from_file(r"C:/Users/Daniel/Desktop/Radwatch/FishSamples2019/background.Spe")
#detector ϵ placeholder = .001

SpeFile: Reading file C:/Users/Daniel/Desktop/Radwatch/FishSamples2019/background.Spe


C:\Users\Daniel\Anaconda3\lib\site-packages\becquerel-0.2.4-py3.7.egg\becquerel\parsers\spectrum_file.py:136: SpectrumFileParsingWarning: Spectrum will be initated without an energy calibration;invalid calibration, energies not monotonically increasing.
  SpectrumFileParsingWarning)


In [5]:
#specs for testing
spec_30 = Spectrum.from_file(r"C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-30min.Spe")
spec_03 = Spectrum.from_file(r"C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-3hr.Spe")
spec_24 = Spectrum.from_file(r"C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-24hr.Spe")

SpeFile: Reading file C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-30min.Spe
SpeFile: Reading file C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-3hr.Spe
SpeFile: Reading file C:\Users\Daniel\Desktop\Radwatch\FishSamples2019\ucb1901-24hr.Spe


In [6]:
specs= [spec_30,spec_03,spec_24]
cal_specs = []
bgspec = am.apply_ecal(bgspec, e_cal)
for i in range(len(specs)):
    cal_specs.append(am.apply_ecal(specs[i],e_cal))

In [ ]:
energies = []
for i in range(len(cal_specs)):
    energies.append(am.get_energies(cal_specs[i])) #peak finding method, bq.peakfinder


In [ ]:
energies[0]

In [ ]:
iso_dfs =[]
#isotope analyzer improvement
for e in energies:
    iso_dfs.append(get_isotopes(e))

In [ ]:
iso_dfs

In [ ]:
print(energies[0])
print(iso_dfs[0].energies)

In [ ]:
nrg_inp = np.array(iso_dfs[0].energies[0])
r,u = am.get_counts(cal_specs[0],bgspec,nrg_inp)
list1 = r/cal_specs[0].livetime
list2 =iso_dfs[0].branching_ratios[0]
products = [a * b for a, b in zip(list1, list2)]
c= np.sum(products)
brbr= iso_dfs[0].branching_ratios[0]
print(products)
print(c/np.sum(brbr))

for k in range(len(cal_specs)):
    for j in iso_dfs:
        for i in range(len(j.energies)):
            c,u = am.get_counts(cal_specs[k],bgspec,i)
            l1 = c/cal_specs[k].livetime
            l2 = j.branch

In [ ]:
spectra_cnts = {"Spectrum 1 Counts":s1cnts,"Spectrum 2 Counts":s2cnts,"Spectrum 3 Counts":s3cnts}
spectra_cnts = pd.DataFrame(spectra_cnts)
spectra_cnts 

In [ ]:
Abr = []
ibrs = []
iAs = []
mbrs =[]
for i in iso_dfs:
    for j in range(len(i.energies)):
        for k in range(len(i.energies[j])):
            ibrs = np.sum(i.branching_ratios[j][k])
            iAs = i.energies[j][k] # want counts/livetime not energies
            mbrs = i.branching_ratios[j][k]
            
print(ibrs)
#print(iAs)

In [ ]:
iso_array = []
for i in iso_dfs:
    for j in i.isotopes:
        if j in iso_array:
           continue 
        else: 
            iso_array.append(j)

In [ ]:
nrg_array = [] #keep energies with isotope? maybe this is where I find the total cps of the isotope and put it in
for i in iso_dfs:
    for j in i.energies:
        if j in nrg_array:
           continue 
        else: 
            nrg_array.append(j)

In [ ]:
iso_npt = {'isotopes':iso_array}#,"energies":nrg_array}   

isotopes = pd.DataFrame(iso_npt)
isotopes

#keep counts associated with isotopes, not just raw energy input
#reimplementing
cnts = []
uncs = []
icnts = []
iuncs = []
spcnt = []
spunc = []
for i in range(len(cal_specs)):
    for j in range(len(iso_dfs[i].energies)):
        icnts, iuncs = am.get_counts(cal_specs[i],bgspec,iso_dfs[i].energies[j])
        spcnt.append(icnts) #how to split arrays for each isotope's energies and keep with each relevant spectra
        spunc.append(iuncs)
        c.append(spcnt)
        u.append(spunc)
        spcnt = []
        spunc = []
    

In [ ]:
print(cnts)
#print(cnts[0])#counts for one iso
#find a way to order them by spectra at least

In [ ]:
#counts = []
#uncertainties =[]
#icounts = []
#iuncertainties = []
#uses PF.PF 
#for i in range(len(cal_specs)):
    icounts, iuncertainties = am.get_counts(cal_specs[i],bgspec,energies[i])
    counts.append(icounts)
    uncertainties.append(iuncertainties)

In [ ]:
print(counts)

In [ ]:
print(cal_specs[0].livetime)

In [ ]:
cps=[]
uncs =[]
for i in range(len(cal_specs)):
    cps.append(counts[i]/cal_specs[i].livetime)
    uncs.append(uncertainties[i]/cal_specs[i].livetime)

In [ ]:
print((cps))

In [ ]:
ϵ = .005
br = 1
A=[]
unc = []
for i in range(len(counts)):
    A.append(am.iso_activity(cps[i],br,ϵ))
    unc.append(am.iso_activity(uncs[i],br,ϵ))

In [ ]:
len(A)

In [ ]:
for i in range(len(A)):
    for j in range(len(A[i])):
        A[i][j] = int(A[i][j])

In [ ]:
print(A)

In [ ]:
t0 = cal_specs[0].start_time.timestamp() - (4*24*60*60) #time of irrad
print(t0)

In [ ]:
dts=[] #time of measure since time of irradiation

for i in range(len(cal_specs)):
    dts.append(am.get_dt(cal_specs[i],t0))
print(dts)

In [ ]:
iso_dfs[0]
#index through the dataframe, get info on each iso
#plug in relevant info to the eq. for each iso
#be able to select iso of interest?
#output all recognized isotopes per sample per measurement?

In [ ]:
flux = 2*10**11 #from davis trigger reactor
Tirrad = 1 *3600 #sec irradiated

#nndc scraping for verified isotope list and 
#get hl,M_A, xs, iso abundance, then list

hl_na24 = 14.9 *3600 #sec
xs_na24 = 0.531399 * (1*10**-24)
M_A = 22.989769 #g/mol
sample_wt = 1 #g
iso_abundance =1 #100% na23